In [61]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from constants import city_list, nba_teams
import preprocessing
import plotly.express as px 
import plotly.graph_objects as go
import plotly.io as pio
pio.templates.default = "plotly_dark"
output_dir = "/Users/joeschlessinger/Documents/jschless.github.io/_includes/assets/conservationofwins"
import os
df = preprocessing.preprocess()
df = df.query('year < 2021')
df.head()

,win_pct,place,sport,team_name,area,year,n_teams
0,0.00,Columbus,NFL,Columbus Tigers,Ohio,1922,1
1,0.00,Evansville,NFL,Evansville Crimson Giants,Chicago,1922,1
2,0.00,Hammond,NFL,Hammond Pros,Ohio,1922,1
3,0.00,Rochester,NFL,Rochester Jeffersons,New York,1922,1
4,0.25,Louisville,NFL,Louisville Colonels,Louisville,1922,1


## Chicago Case Study

In [62]:
sports = set(["nfl", "mlb", "nba"])
city = "Chicago"
chicago = df.query(f"area == '{city}' and year > 1920").groupby("year").win_pct.mean().reset_index()
def temp(df):
    acc = ""
    for i, row in df.iterrows():
        acc += row.team_name + ": " + str(row.win_pct)[:5] + "<br>"
    return acc

blah = chicago.merge(df.query("area == 'Chicago'").groupby(["year"]).apply(temp).reset_index(), on="year")

fig = go.Figure(go.Scatter(
    x = blah.year.tolist(),
    y = blah.win_pct.tolist(),
    text= blah[0].tolist(),
    hovertemplate =
    '<br><b>Year</b>: %{x}<br>'+
    '<b>Winning Pct.</b>: %{y:.2f}<br>'+
    '%{text}' +
    '<extra></extra>'
    ))
fig.update_layout(
    title="Average Winning Percentage for Chicago Sports Teams",
    xaxis_title="Year",
    yaxis_title="Winning Percentage",
)
fig.add_hline(y=.5, line_width=1, line_dash="dash", line_color="white")
fig.write_html(os.path.join(output_dir,"chicago_avg.html"))
fig.show()

In [21]:
print(f"Total Seasons:", chicago.query("year > 1940").count())
print(f"Seasons with winning percentage above .500:", chicago.query("year > 1940 and win_pct >= .5").count())

Total Seasons: year       80
win_pct    80
dtype: int64
Seasons with winning percentage above .500: year       36
win_pct    36
dtype: int64


## Visualizing Multiple Cities Together

In [63]:
def temp(df):
    return "".join([row.team_name + ": " + str(row.win_pct)[:5] + "<br>" for i, row in df.iterrows()])
temp_df = df.query('year > 1920').groupby(["year", "place"]).win_pct.mean().reset_index()
temp_df = temp_df.merge(df.groupby(["year", "place"]).apply(temp).reset_index(), on=["year", "place"]).rename(columns={0: "teams"})
cities = ["Chicago", "New York", "Boston", "Los Angeles", "Atlanta"]
fig = go.Figure()
for city in cities:
    temp = temp_df.query(f"place == '{city}'")
    win_pct = str(temp.win_pct.mean())[:5]
    print(city, win_pct)
    fig.add_trace(
        go.Scatter(x=temp.year.tolist(), 
        y=temp.win_pct.tolist(),
        text=temp.teams.tolist(),
        mode="lines",
        hovertemplate =
        '<br><b>Year</b>: %{x}<br>'+
        '<b>Winning Pct.</b>: %{y:.2f}<br>'+
        '%{text}' +
        '<extra></extra>',
        name=city + f" ({win_pct})"))
fig.add_hline(y=.5, line_width=1, line_dash="dash", line_color="white")
fig.update_layout(
    title= "Winning Percentage by City Over Time",
    xaxis_title="Year",
    yaxis_title="Winning Percentage",
    legend_title_text="City (Lifetime Winning Pct.)"
)
fig.write_html(os.path.join(output_dir,"city_avg.html"))
fig.show()


Chicago 0.507
New York 0.506
Boston 0.513
Los Angeles 0.500
Atlanta 0.478


## Breakdown By Sport

In [43]:
city = "Chicago"
temp = df.query(f"area == '{city}'").groupby(["team_name", "year"]).win_pct.mean().reset_index()
teams = ['Chicago Bears', 'Chicago Blackhawks',
       'Chicago Bulls', 'Chicago Cubs','Chicago White Sox']
fig = go.Figure()
for t in teams: 
    win_pct = str(temp.query(f"team_name == '{t}'").win_pct.mean())[:5]
    fig.add_trace(go.Scatter(x=temp.query(f"team_name == '{t}'").year.tolist(), 
    y=temp.query(f"team_name == '{t}'").win_pct.tolist(),
                        mode='lines',
                        name=t+ f" ({win_pct})"))
    
    print("average winning percentage for", t, win_pct)

fig.update_layout(
    title= "Winning Percentage of Chicago Teams Over Time",
    xaxis_title="Year",
    yaxis_title="Winning Percentage",
    legend_title_text="Team (Lifetime Winning Pct.)"
)
fig.add_hline(y=.5, line_width=1, line_dash="dash", line_color="white")

fig.write_html(os.path.join(output_dir,"chicago_by_sport.html"))
fig.show()


average winning percentage for Chicago Bears 0.576
average winning percentage for Chicago Blackhawks 0.441
average winning percentage for Chicago Bulls 0.520
average winning percentage for Chicago Cubs 0.507
average winning percentage for Chicago White Sox 0.502


### Exploring Cities with 3 + teams 

In [65]:
teams_per_place = df.groupby(["year", "place"]).team_name.count()
teams_3 = teams_per_place[teams_per_place > 2].index
def temp(df):
    acc = ""
    for i, row in df.iterrows():
        acc += row.team_name + ": " + str(row.win_pct)[:5] + "<br>"
    return acc
teams = df.groupby(["year", "place"]).apply(temp)

In [66]:
temp = df.groupby(["year", "place"]).win_pct.mean()[teams_3].reset_index()
temp = temp.set_index(["year", "place"])
temp['teams'] = teams
temp = temp.reset_index()
temp
#temp[temp.year > 1950].sort_values("win_pct").head(30)

,year,place,win_pct,teams
0,1922,Chicago,0.589827,Chicago Bears: 0.75<br>Chicago White Sox: 0.5<...
1,1923,Chicago,0.601671,Chicago Bears: 0.818<br>Chicago White Sox: 0.4...
2,1923,St. Louis,0.401061,St. Louis All-Stars: 0.2<br>St. Louis Browns: ...
3,1924,Chicago,0.605928,Chicago Bears: 0.857<br>Chicago White Sox: 0.4...
4,1925,Boston,0.533242,Boston Bulldogs: 0.833<br>Boston Red Sox: 0.30...
...,...,...,...,...
872,2020,Minnesota,0.515082,Minnesota Vikings: 0.438<br>Minnesota Twins: 0...
873,2020,New York,0.421102,New York Jets: 0.125<br>New York Giants: 0.375...
874,2020,Philadelphia,0.447290,Philadelphia Eagles: 0.281<br>Philadelphia Phi...
875,2020,Pittsburgh,0.548792,Pittsburgh Steelers: 0.75<br>Pittsburgh Pirate...


In [68]:
blah = temp[temp.year>1950]
fig = go.Figure(go.Scatter(
    x = temp.year.tolist(),
    y = temp.win_pct.tolist(),
    text = temp.teams.tolist(),
    mode="markers",
    hovertemplate =
    '<br><b>Year</b>: %{x}<br>'+
    '<b>Winning Pct.</b>: %{y:.2f}<br>'+
    '%{text}' +
    '<extra></extra>')
    )
fig.update_layout(
    title="Average Winning Percentage for Sports Locales Over Time",
    xaxis_title="Year",
    yaxis_title="Winning Percentage",
    # legend_title="Legend Title",
    # font=dict(
    #     family="Courier New, monospace",
    #     size=18,
    #     color="RebeccaPurple"
    # )
)
fig.add_hline(y=.5, line_width=.5, line_dash="dash", line_color="white")
fig.write_html(os.path.join(output_dir,"season_scatter.html"))
fig.show()

Almost every season is in the .4-.6 range. The truly exceptional (and unexceptional)

In [60]:
print(temp.sort_values('win_pct', ascending=False).head(10).to_markdown(index=False))

|   year | place        |   win_pct | teams                                                                                                      |
|-------:|:-------------|----------:|:-----------------------------------------------------------------------------------------------------------|
|   1927 | New York     |  0.733563 | New York Giants: 0.889<br>New York Giants: 0.597<br>New York Yankees: 0.714<br>                            |
|   1998 | Atlanta      |  0.71644  | Atlanta Falcons: 0.875<br>Atlanta Braves: 0.654<br>Atlanta Hawks: 0.62<br>                                 |
|   1972 | Boston       |  0.689898 | Boston Red Sox: 0.548<br>Boston Celtics: 0.829<br>Boston Bruins: 0.692<br>                                 |
|   1937 | Chicago      |  0.687446 | Chicago Bears: 0.9<br>Chicago White Sox: 0.558<br>Chicago Cubs: 0.603<br>                                  |
|   1926 | Chicago      |  0.687293 | Chicago Bears: 1.0<br>Chicago White Sox: 0.529<br>Chicago Cubs: 0.532<br>       

In [58]:
print(temp.sort_values('win_pct').head(10).to_markdown(index=False))

|   year | place        |   win_pct | teams                                                                                                                    |
|-------:|:-------------|----------:|:-------------------------------------------------------------------------------------------------------------------------|
|   1971 | Buffalo      |  0.215564 | Buffalo Bills: 0.071<br>Buffalo Braves: 0.268<br>Buffalo Sabres: 0.307<br>                                               |
|   1972 | Philadelphia |  0.250135 | Philadelphia Eagles: 0.179<br>Philadelphia Phillies: 0.378<br>Philadelphia 76ers: 0.11<br>Philadelphia Flyers: 0.333<br> |
|   1939 | Philadelphia |  0.253285 | Philadelphia Eagles: 0.1<br>Philadelphia Athletics: 0.361<br>Philadelphia Phillies: 0.298<br>                            |
|   1940 | Philadelphia |  0.256149 | Philadelphia Eagles: 0.091<br>Philadelphia Athletics: 0.350<br>Philadelphia Phillies: 0.326<br>                          |
|   1936 | Philadelphia |  0.26001

### Largest Improvements and Declines

# Winning PCT

In [8]:
fig = px.histogram(df, x="win_pct", nbins=30)
fig.update_layout(
    title="Histogram of Winning Percentages",
    xaxis_title="Winning Percentage",
    yaxis_title="# Seasons",
)
fig.write_image(os.path.join(output_dir,"hist.jpeg"))
print(f"Mean win pct is {df.win_pct.mean()}")
print(f"Std dev is {df.win_pct.std()}")

Mean win pct is 0.4883278372078791
Std dev is 0.15160039855772944


## Bokeh Deez Nuts

In [41]:
from bokeh.layouts import column
from bokeh.models import ColumnDataSource, CustomJS, Slider, MultiChoice
from bokeh.plotting import figure, show

x = [x*0.005 for x in range(0, 200)]
y = x

source = ColumnDataSource.from_df(df)

plot = figure(width=400, height=400)

plot.scatter('year', 'win_pct', source=source)

options = df.place.unique().tolist()
multi_choice = MultiChoice(value=["Chicago"], options=options)

callback = CustomJS(args=dict(source=source), code="""
    const places = cb_obj.value
    console.log(source)
    const new_data = source.filter(x => places.includes(x.place))
    const x = new_data.year
    const y = new_data.win_pct
    source = { x, y }
""")

multi_choice.js_on_change("value", callback)
# slider = Slider(start=0.1, end=4, value=1, step=.1, title="power")
# slider.js_on_change('value', callback)

layout = column(multi_choice, plot)
show(layout)


In [42]:
print(source)

{'index': array([   0,    1,    2, ..., 7422, 7423, 7424]), 'win_pct': array([0.        , 0.        , 0.        , ..., 0.32142857, 0.33928571,
       0.30357143]), 'place': array(['Columbus', 'Evansville', 'Hammond', ..., 'Columbus', 'New Jersey',
       'Anaheim'], dtype=object), 'sport': array(['NFL', 'NFL', 'NFL', ..., 'NHL', 'NHL', 'NHL'], dtype=object), 'team_name': array(['Columbus Tigers', 'Evansville Crimson Giants', 'Hammond Pros',
       ..., 'Columbus Blue Jackets', 'New Jersey Devils', 'Anaheim Ducks'],
      dtype=object), 'area': array(['Ohio', 'Chicago', 'Ohio', ..., 'Ohio', 'New York', 'Los Angeles'],
      dtype=object), 'year': array([1922, 1922, 1922, ..., 2021, 2021, 2021]), 'n_teams': array([1, 1, 1, ..., 1, 1, 1])}


In [36]:
from bokeh.io import show
from bokeh.models import CustomJS, MultiChoice

OPTIONS = ["foo", "bar", "baz", "quux"]

multi_choice = MultiChoice(value=["foo", "baz"], options=OPTIONS)
multi_choice.js_on_change("value", CustomJS(code="""
    console.log('multi_choice: value=' + this.value, this.toString())
"""))

show(multi_choice)